In [30]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI


## Load data

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader


loader = TextLoader(
    "data/text/O6_1A_Umweltgerechte_und_biodiversitaetsfoerdernde_Bewirtschaftung_(UBB)_2023_04.txt")
documents = loader.load()



# split document into sentences
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

docs = text_splitter.split_documents(documents)


## Import sentence embeddings & init vectorstore and retriever

In [38]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")

vectorstore = FAISS.from_documents(docs, embeddings)

retriever = vectorstore.as_retriever()


In [39]:
template = """Beantworte die Frage mit den gegebenen Informationen:
{context}

Frage: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()


In [42]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

res = chain.invoke(
    "Was ist auf der Fläche nicht erlaubt?")
print(res)


Auf der Fläche sind folgende Aktivitäten nicht erlaubt: Beweidung nach dem 15. September, Einsatz von Pflanzenschutzmitteln und jegliche Düngung vom 1. Januar des Jahres der ersten Angabe des Schlages als Biodiversitätsfläche bis zum Umbruch oder anderweitiger Deklaration der Flächen, Befahren und Düngung der Fläche nach dem 15. August bis zur nächsten Nutzung im Folgejahr.


In [40]:
questions = ["Für welche Art von Flächen ist die Prämie?",
             "Was ist UBB?",
             "Was ist auf der Fläche nicht erlaubt?"]


In [36]:



vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=embeddings)

retriever = vectorstore.as_retriever()
